In [1]:
import os
from deepClassifier.constants import *

In [2]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size : list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [16]:
from pathlib import Path 

class ConfigurationManager:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH, 
                params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        try:
            config = self.config.prepare_base_model
            params = self.params

            root_dir = config.root_dir
            create_directories([root_dir])
            base_model_path = config.base_model_path
            updated_base_model_path = config.updated_base_model_path

            params_image_size = params.IMAGE_SIZE
            params_learning_rate = params.LEARNING_RATE
            params_include_top = params.INCLUDE_TOP
            params_weights = params.WEIGHTS
            params_classes = params.CLASSES

            prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=Path(root_dir),
                base_model_path=Path(base_model_path),
                updated_base_model_path=Path(updated_base_model_path),
                params_image_size=params_image_size,
                params_learning_rate=params_learning_rate,
                params_include_top=params_include_top,
                params_weights=params_weights,
                params_classes=params_classes
            )
            
            return prepare_base_model_config
        except Exception as e:
            raise e

In [36]:
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        try: 
            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape = self.config.params_image_size,
                weights = self.config.params_weights,
                include_top = self.config.params_include_top             #Do not include last layer
            )

            base_model_path = self.config.base_model_path

            self.save_model(path = base_model_path, model = self.model)

        except Exception as e:
            raise e
        
    @staticmethod
    def _prepare_final_model(model, classes, freeze_all, freeze_till, learning_rate):
        try:
            if freeze_all:
                for layer in model.layers:
                    model.trainable = False 
            elif (freeze_till is not None) and (freeze_till>0):
                for layer in model.layers[:-freeze_till]:
                    model.trainable = False

            flatten_in = tf.keras.layers.Flatten()(model.output)
            prediction = tf.keras.layers.Dense(
                units = classes,
                activation = 'softmax'
            )(flatten_in)

            final_model = tf.keras.models.Model(
                inputs = model.input,
                outputs = prediction
            )

            final_model.compile(
                optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy']
            )
            final_model.summary()
            return final_model
        except Exception as e:
            raise e
        
        
    def update_base_model(self):
        self.full_model = self._prepare_final_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [37]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e 

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   